# The Model

## setup
Import, load data and instantiate relevant classes (DataLoader, Dataset etc)

In [30]:
from importlib import reload

In [7]:
## imports
#  modules
import numpy as np
import pandas as pd
import xarray as xr
import warnings
import os

import torch
from torch import nn

# custom code
from dask_addons import FlattenedDaskDataset
from neural_net import DNN, geoDataset, BatchDataLoader, train, test
from open_fire_data import FlattenedTruthTable

warnings.filterwarnings('ignore')

In [8]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:62590/status,
Dashboard: http://127.0.0.1:62590/status,Workers: 4
Total threads: 8,Total memory: 15.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62591,Workers: 0
Dashboard: http://127.0.0.1:62590/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:62619,Total threads: 2
Dashboard: http://127.0.0.1:62620/status,Memory: 3.88 GiB
Nanny: tcp://127.0.0.1:62594,


In [9]:
DATA_DIR_PATH = '../data/_ZARR_FILES'

data_path = os.path.join(DATA_DIR_PATH, 'alaska_full.zarr')
prior_data_path = os.path.join(DATA_DIR_PATH, 'alaska_prior.zarr')

data = xr.open_zarr(data_path, decode_timedelta=False)
prior_data = xr.open_zarr(prior_data_path, decode_timedelta=False)

In [10]:
input_ds = FlattenedDaskDataset(data, prior_data)
input_ds.setup()

[compute_running_totals] run time: 0.1734788417816162s


In [37]:
import open_fire_data as ofd
reload(ofd)
from open_fire_data import FlattenedTruthTable

In [38]:
fire_data = FlattenedTruthTable(
    pd.read_csv('../data/_FIRE/alaska_range_csv/data.csv'),
    lat_vals = input_ds.latitude,
    long_vals = input_ds.longitude
)

In [39]:
ds = geoDataset(input_ds, fire_data, feature_num = 13)

## training


In [13]:
model = DNN()
print(model)

DNN(
  (network): Sequential(
    (0): Linear(in_features=13, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [40]:
train_dataloader = BatchDataLoader(ds)
test_dataloader = BatchDataLoader(ds)

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [35]:
import neural_net as nen
reload(nen)

from neural_net import test, train

In [41]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, device)
    test(test_dataloader, model, loss_fn, device)
print("Done!")

Epoch 1
-------------------------------


ValueError: not enough values to unpack (expected 2, got 1)